##importing libraries##

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats
from scipy.stats import percentileofscore 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [2]:
stocks=pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


removing the tickrs of stocks that have been delisted

In [3]:
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW','AET','ADS','ANDV','BHGE','APTV'])]
# stocks = stocks[~stocks['Ticker'].isin(['AET','ADS','ANDV','BHGE','APTV'])]


In [20]:
stocks.reset_index(inplace=True,drop=True)
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
495,YUM
496,ZBH
497,ZBRA
498,ZION


##gupt_gyan is the secrets.py file and i renamed it as the notebook compiler was confusing it with some numpy necessaries##

In [4]:
from gupt_gyan import IEX_CLOUD_API_TOKEN

##first we go through the whole process and make a simple model##

##now we parse the api's in batches as it reduces the loading time##

In [5]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
#print(len(symbol_groups))
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])
print(symbol_strings)
#print(symbol_groups)
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA,CME', 'CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD,GIS,GL', 'GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILMN,INCY,

In [7]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
hqm_dataframe.replace(to_replace=[None], value=0, inplace=True)
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

In [8]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col=f'{time_period} Price Return'
        percentile_col= f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0       0.52
1      0.194
2      0.458
3      0.694
4      0.898
       ...  
495    0.604
496    0.286
497     0.02
498    0.402
499     0.21
Name: One-Year Return Percentile, Length: 500, dtype: object
0      0.894
1      0.114
2      0.508
3      0.494
4      0.658
       ...  
495     0.69
496    0.422
497     0.03
498    0.386
499    0.296
Name: Six-Month Return Percentile, Length: 500, dtype: object
0      0.774
1       0.58
2       0.49
3       0.41
4      0.742
       ...  
495    0.472
496    0.508
497    0.038
498    0.488
499    0.156
Name: Three-Month Return Percentile, Length: 500, dtype: object
0      0.766
1       0.79
2      0.888
3      0.446
4      0.536
       ...  
495    0.576
496     0.37
497    0.018
498    0.206
499    0.256
Name: One-Month Return Percentile, Length: 500, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,137.940,N/A,-0.103669,0.52,0.183159,0.894,0.075523,0.774,0.168525,0.766,N/A
1,AAL,14.441,N/A,-0.301567,0.194,-0.247399,0.114,0.003630,0.58,0.176317,0.79,N/A
2,AAP,185.290,N/A,-0.144351,0.458,-0.041016,0.508,-0.021763,0.49,0.210966,0.888,N/A
3,AAPL,145.340,N/A,0.009888,0.694,-0.044790,0.494,-0.058039,0.41,0.091894,0.446,N/A
4,ABBV,145.230,N/A,0.317746,0.898,0.013342,0.658,0.059853,0.742,0.106690,0.536,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
495,YUM,124.818,N/A,-0.034117,0.604,0.030775,0.69,-0.029202,0.472,0.115542,0.576,N/A
496,ZBH,108.520,N/A,-0.238623,0.286,-0.072106,0.422,-0.017316,0.508,0.069894,0.37,N/A
497,ZBRA,231.714,N/A,-0.602997,0.02,-0.386322,0.03,-0.263600,0.038,-0.093936,0.018,N/A
498,ZION,48.820,N/A,-0.174099,0.402,-0.084722,0.386,-0.021864,0.488,0.015110,0.206,N/A


In [9]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

In [10]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:50]

In [12]:
portfolio_size =10000000

In [13]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,137.940,1449,-0.103669,0.52,0.183159,0.894,0.075523,0.774,0.168525,0.766,0.7385
1,AAL,14.441,13849,-0.301567,0.194,-0.247399,0.114,0.003630,0.58,0.176317,0.79,0.4195
2,AAP,185.290,1079,-0.144351,0.458,-0.041016,0.508,-0.021763,0.49,0.210966,0.888,0.586
3,AAPL,145.340,1376,0.009888,0.694,-0.044790,0.494,-0.058039,0.41,0.091894,0.446,0.511
4,ABBV,145.230,1377,0.317746,0.898,0.013342,0.658,0.059853,0.742,0.106690,0.536,0.7085
5,ABC,161.800,1236,0.277108,0.886,0.055997,0.742,0.098662,0.81,0.158889,0.726,0.791
6,ABMD,385.110,519,0.082696,0.764,0.332277,0.968,0.295738,0.966,0.538961,1.0,0.9245
7,ABT,98.640,2027,-0.209505,0.34,-0.117175,0.314,-0.083895,0.33,0.031949,0.252,0.309
8,ACN,269.600,741,-0.211484,0.336,-0.070022,0.428,-0.067313,0.394,0.099024,0.494,0.413
9,ADBE,301.900,662,-0.511275,0.036,-0.235012,0.13,-0.233809,0.05,0.150667,0.698,0.2285


In [14]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [15]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [16]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [17]:
writer.save()